# Coupling to Ideal Loads
In this notebook, we investigate the WEST ICRH antenna behaviour when the front-face is considered as the combination of ideal (and independant) loads made of impedances all equal to $Z_s=R_c+j X_s$, where $R_c$ corresponds to the coupling resistance and $X_s$ is the strap reactance.  

<img src="West_front_face_ideal.png" width="300"/>

In such case, the power delivered to the plasma/front-face is then:

$$
P_t 
= \frac{1}{2} \sum_{i=1}^4 \Re[V_i I_i^* ] 
= \frac{1}{2} \sum_{i=1}^4 \Re[Z_i] |I_i|^2
= \frac{1}{2} R_c \sum_{i=1}^4 |I_i|^2
$$
Hence, we have defined the coupling resistance as:

$$
R_c = \frac{\sum_{i=1}^4 \Re[Z_i] |I_i|^2}{\sum_{i=1}^4 |I_i|^2}
$$

Inversely, the coupling resistance can be determine from:

$$
R_c = \frac{2 P_t}{\sum_{i=1}^4 |I_i|^2}
$$

In practice however, it is easier to measure RF voltages than currents. 

$$
I = \frac{V}{Z_s} = \frac{V}{R_c + j X_s} 
\rightarrow 
|I|^2 = \frac{|V|^2}{|R_c + j X_s|}
\approx
\frac{|V|^2}{|X_s|^2}
$$
since in $|X_s|>>|R_c|$.

The antenna model allows to calculate the coupling resistance from currents (`.Rc()` method) or from the voltage (`.Rc_WEST()` method).

The strap reactance $X_s$ depends on the strap geometry and varies with the frequency. So, let's find how the strap reactance from the realistic CAD model. 

In [1]:
%matplotlib widget

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import skrf as rf
from tqdm.notebook import tqdm

# WEST ICRH Antenna package
import sys; sys.path.append('..')
from west_ic_antenna import WestIcrhAntenna

# styling the figures
rf.stylely()

C:\Users\JH218595\Documents\scikit-rf\skrf\plotting.py:1441: UserWarning: Style includes a parameter, 'interactive', that is not related to style.  Ignoring
  mpl.style.use(os.path.join(pwd, style_file))


## Coupling to an ideal front-face
Coupling to an ideal front face of coupling resistance $R_c$ is easy using the the `.load()` method of the `WestIcrhAntenna` class. This method takes into account the strap reactance frequency fit (derived in [Strap Reactance Frequency Fit](./strap_reactance.ipynb))   

In [3]:
freq = rf.Frequency(30, 70, npoints=1001, unit='MHz')
ant_ideal = WestIcrhAntenna(frequency=freq)
ant_ideal.load(Rc=1) # 1 Ohm coupling resistance front-face

In [4]:
# matching left and right sides : note that the solutions are (almost) the same
f_match = 55.5e6
C_left = ant_ideal.match_one_side(f_match=f_match, side='left')
C_right = ant_ideal.match_one_side(f_match=f_match, side='right')

True solution #1: [53.56567365 45.98820709]
True solution #1: [53.56564882 45.98822849]


At the difference of the "real" situation (see the [Matching](./matching.ipynb) or the [Coupling to a TOPICA plasma](./coupling_to_plasma_from_TOPICA.ipynb)), here is no poloidal neither toroidal coupling of the straps in this front-face model. This leads to:
* Match soluitions are the same for both sides (within $10^{-3}$ pF). 
* Using the match solutions for each sides does not require to shift the operating frequency:

In [5]:
# dipole excitation
power = [1, 1]
phase = [0, rf.pi]

# active S-parameter for the match point:
C_match = [C_left[0], C_left[1], C_right[2], C_right[3]]
s_act = ant_ideal.s_act(power, phase, Cs=C_match)

fig, ax = plt.subplots()
ax.plot(ant_ideal.f_scaled, 20*np.log10(np.abs(s_act)), lw=2)
ax.legend(('$S_{act,1}$', '$S_{act,2}$'))
ax.grid(True)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Match Points vs Coupling Resistance

Let's determine the match points for a range of coupling resistance at a given frequency

In [6]:
f_match = 55e6
Rcs = np.r_[0.01, 0.05, np.arange(0.1, 2.5, 0.2)]

C_matchs = []
ant = WestIcrhAntenna()
for Rc in tqdm(Rcs):
    ant.load(Rc)
    C_match = ant.match_one_side(f_match=f_match)
    C_matchs.append(C_match)
    

Wrong solution (out of range capacitor) ! Re-doing...
False solution #1: [150. 150.]
True solution #1: [52.02785554 50.46880396]
True solution #1: [52.4097308  50.09882462]
True solution #1: [52.7552753  49.76713493]
Wrong solution (out of range capacitor) ! Re-doing...
False solution #1: [150. 150.]
True solution #1: [53.66611397 48.90006135]
Wrong solution (out of range capacitor) ! Re-doing...
False solution #1: [150. 150.]
Wrong solution (out of range capacitor) ! Re-doing...
False solution #1: [150. 150.]
Wrong solution (out of range capacitor) ! Re-doing...
False solution #1: [150. 150.]
True solution #1: [54.2767751 48.3126251]
True solution #1: [54.73987025 47.84863587]
True solution #1: [55.102375   47.45603461]
Wrong solution (out of range capacitor) ! Re-doing...
False solution #1: [150. 150.]
True solution #1: [55.38480972 47.10756983]
Wrong solution (out of range capacitor) ! Re-doing...
False solution #1: [150. 150.]
Wrong solution (out of range capacitor) ! Re-doing...
F

As the coupling resistance increases, the distance between capacitances (Top vs Bottom) increases: 

In [7]:
fig, ax = plt.subplots()
ax.plot(Rcs, np.array(C_matchs)[:,0:2], lw=2, marker='o')
ax.axhline(C_matchs[0][0], ls='--', color='C0')
ax.axhline(C_matchs[0][1], ls='--', color='C1')
ax.set_xlabel('Rc [Ohm]')
ax.set_ylabel('C [pF]')
ax.legend(('Top', 'Bot'))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Displayed differently, the distance between capacitances (Top - Bottom) versus coupling resistance is:

In [10]:
delta_C_pos =  np.array(C_matchs)[:,0] - C_matchs[0][0]
delta_C_neg =  C_matchs[0][1] - np.array(C_matchs)[:,1]

fig, ax = plt.subplots()
ax.plot(Rcs, delta_C_pos, label='Top: + $\Delta C$', lw=2)
ax.plot(Rcs, delta_C_neg, label='Bot: - $\Delta C$', lw=2)
ax.set_xlabel('Rc [Ohm]')
ax.set_ylabel('$\Delta C$ [pF]')
ax.set_ylim(bottom=0)
ax.set_xlim(left=0)
ax.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Load Resilience Curves
Ideal loads is usefull to study the behaviour of the load tolerance property of the antenna and the capacitance match points. It is only necessary to work on half-antenna here, because there is no coupling between toroidal elements. 

Now that we have figured out the match points, let's vary the coupling resistances for a fixed match point and look to the return power (or VSWR): this will highlight the load resilience property of the antenna.

In [13]:
# create a single frequency point antenna to speed-up calculations
ant = WestIcrhAntenna(frequency=rf.Frequency.from_f(f_match, unit='Hz'))

fig, ax = plt.subplots()
power = [1, 1]
phase = [0, np.pi]

for C_match in tqdm(C_matchs[0:8]):
    SWRs = []
    ant.Cs = [C_match[0], C_match[1], 150, 150]
    for Rc in Rcs:
        ant.load(Rc)
        SWR = ant.circuit().network.s_vswr.squeeze()[0,0]
        SWRs.append(SWR)
    ax.plot(Rcs, np.array(SWRs), lw=2)

ax.set_xlabel('Rc [Ohm]')
ax.set_ylabel('VSWR')
ax.set_ylim(1, 8)
ax.axhline(2, color='r')
ax.legend(Rcs)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [12]:
from IPython.core.display import HTML
def _set_css_style(css_file_path):
    """
    Read the custom CSS file and load it into Jupyter
    Pass the file path to the CSS file
    """
    styles = open(css_file_path, "r").read()
    s = '<style>%s</style>' % styles
    return HTML(s)

_set_css_style('custom.css')